In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 작업 디렉토리 변경
import os
print(os.getcwd(), end=' -> ')
os.chdir('/content/drive/MyDrive/Colab Notebooks/25-1pattern_recognition')
print(os.getcwd())

/content -> /content/drive/MyDrive/Colab Notebooks/25-1pattern_recognition




---



In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, make_scorer
from scipy.stats.mstats import winsorize

# preprocessing

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# CSV 파일 불러오기
df = pd.read_csv('test.csv')
df.head()

,id,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,data_channel,weekday
0,4979,6.0,486.0,0.579832,1.0,0.747573,NaN,0.0,1.0,0.0,...,0.5,-0.260000,-0.5500,-0.100000,0.0,0.000000,0.500000,0.000000,World,Tuesday
1,15552,12.0,422.0,0.497537,1.0,0.618868,NaN,0.0,NaN,0.0,...,0.7,-0.325000,-1.0000,-0.125000,NaN,0.000000,0.500000,0.000000,Business,Tuesday
2,29370,7.0,374.0,0.549598,NaN,0.697674,7.0,4.0,NaN,0.0,...,0.6,NaN,NaN,-0.100000,0.0,0.000000,0.500000,0.000000,Tech,Thursday
3,37272,NaN,181.0,0.720670,1.0,0.783333,10.0,NaN,1.0,NaN,...,0.4,-0.185714,-0.3000,-0.071429,0.0,0.000000,0.500000,0.000000,Entertainment,Monday
4,6836,NaN,299.0,0.602740,1.0,0.769663,2.0,1.0,1.0,NaN,...,0.5,-0.204167,-0.3125,-0.100000,NaN,0.068182,0.227273,0.068182,Entertainment,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9515 entries, 0 to 9514
Data columns (total 47 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            9515 non-null   int64  
 1   n_tokens_title                8562 non-null   float64
 2   n_tokens_content              8540 non-null   float64
 3   n_unique_tokens               8574 non-null   float64
 4   n_non_stop_words              8554 non-null   float64
 5   n_non_stop_unique_tokens      8600 non-null   float64
 6   num_hrefs                     8571 non-null   float64
 7   num_self_hrefs                8570 non-null   float64
 8   num_imgs                      8587 non-null   float64
 9   num_videos                    8485 non-null   float64
 10  average_token_length          8540 non-null   float64
 11  num_keywords                  8583 non-null   float64
 12  kw_min_min                    8591 non-null   float64
 13  kw_

In [10]:
df.describe()

,id,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
count,9515.000000,8562.000000,8540.000000,8574.000000,8554.000000,8600.000000,8571.000000,8570.000000,8587.000000,8485.000000,...,8543.000000,8538.00000,8593.000000,8580.000000,8614.000000,8499.000000,8548.000000,8592.000000,8549.000000,8519.000000
mean,19733.341566,10.417309,549.788056,0.612038,0.969956,0.748137,10.919963,3.278180,4.679166,1.238421,...,0.353301,0.09562,0.751536,-0.258501,-0.519224,-0.108191,0.281562,0.071585,0.340240,0.156961
std,11439.787304,2.114024,490.588066,7.566031,0.170720,7.003546,11.776562,3.687286,8.662910,4.275307,...,0.103888,0.07058,0.248522,0.126710,0.289854,0.095191,0.322073,0.268164,0.189661,0.226798
min,11.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000
25%,9803.500000,9.000000,249.000000,0.471690,1.000000,0.625000,4.000000,1.000000,1.000000,0.000000,...,0.306692,0.05000,0.600000,-0.327516,-0.700000,-0.125000,0.000000,0.000000,0.166667,0.000000
50%,19546.000000,10.000000,410.000000,0.539776,1.000000,0.689189,8.000000,3.000000,1.000000,0.000000,...,0.357876,0.10000,0.800000,-0.253157,-0.500000,-0.100000,0.150000,0.000000,0.500000,0.000000
75%,29648.500000,12.000000,712.250000,0.608059,1.000000,0.756219,14.000000,4.000000,4.000000,1.000000,...,0.410559,0.10000,1.000000,-0.186068,-0.300000,-0.050000,0.500000,0.150000,0.500000,0.250000
max,39642.000000,20.000000,8474.000000,701.000000,1.000000,650.000000,304.000000,56.000000,111.000000,75.000000,...,1.000000,1.00000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,1.000000


## 데이터 전처리

In [13]:
# 많은 값들이 같은 값을 가지므로 삭제한다
df.drop(columns=['kw_min_min'], axis=1, inplace=True)
df.drop(columns=['kw_max_max'], axis=1, inplace=True)
df.drop(columns=['n_non_stop_words'], axis=1, inplace=True)

In [14]:
# 'id'는 각 샘플마다 고유한 값을 가진다. 학습에 사용할 값이 아니므로 삭제한다
df.drop(columns=['id'], axis=1, inplace=True)

In [16]:
# 결측치 처리
for col in df.columns:
    if df[col].dtype == 'object': # 범주형 → 최빈값
        df[col] = df[col].fillna(df[col].mode()[0]) # 첫번째 최빈값
    else: # 숫자형 → 중간값
        df[col] = df[col].fillna(df[col].median())

In [17]:
# is_weekend 변수 추가
df['is_weekend'] = df['weekday'].isin(['Saturday', 'Sunday']).astype(int)

# 범주형 데이터 One-Hot Encoding
df = pd.get_dummies(df, columns=['data_channel', 'weekday'], drop_first=True)



---



## 전처리 (train, val 나누기) (정규화)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9515 entries, 0 to 9514
Data columns (total 53 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   n_tokens_title                9515 non-null   float64
 1   n_tokens_content              9515 non-null   float64
 2   n_unique_tokens               9515 non-null   float64
 3   n_non_stop_unique_tokens      9515 non-null   float64
 4   num_hrefs                     9515 non-null   float64
 5   num_self_hrefs                9515 non-null   float64
 6   num_imgs                      9515 non-null   float64
 7   num_videos                    9515 non-null   float64
 8   average_token_length          9515 non-null   float64
 9   num_keywords                  9515 non-null   float64
 10  kw_max_min                    9515 non-null   float64
 11  kw_avg_min                    9515 non-null   float64
 12  kw_min_max                    9515 non-null   float64
 13  kw_

In [19]:
X_test = df
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# test

In [20]:
import joblib
stacking_model = joblib.load('best_stacking_model.pkl')

In [21]:
y_test_prob = stacking_model.predict_proba(X_test)[:, 1]

# 클래스 예측
y_test_pred = (y_test_prob > 0.5).astype(int)

# 테스트 데이터에 예측 결과 추가
df['Prediction'] = y_test_pred

# 결과 저장
df[['Prediction']].to_csv('test_predictions.csv', index=False)

[LightGBM] [Warning] feature_fraction is set=0.8488251044495548, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8488251044495548
